In [1]:
# dataDir = "/home/ababil/BUET/AV/csv"
dataDir = "D:/AV/Code/location-2"
# dataDir = "D:/AV/DataSetExploration/drone-dataset-tools/data/location-2"

In [2]:
exec(open('init_notebook.py').read())

current working dirD:\AV\Code\behavior-hypotheses\src


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import math

from tti_dataset_tools.TrajectoryTransformer import TrajectoryTransformer
from tti_dataset_tools.TrajectoryVisualizer import TrajectoryVisualizer
from tti_dataset_tools.TrajectoryCleaner import TrajectoryCleaner
from tti_dataset_tools.ColMapper import ColMapper

In [5]:
sceneId = 210
sceneId2 = 211
date = "2023-05-04"
fps=10
minYDisplacement = 5
maxXDisplacement = 6
pedDf = pd.read_csv(os.path.join(dataDir, f"{date}-fps-{fps}-scene-{sceneId}-pedestrians.csv"))
pedDf2 = pd.read_csv(os.path.join(dataDir, f"{date}-fps-{fps}-scene-{sceneId2}-pedestrians.csv"))
# otherDf = pd.read_csv(os.path.join(dataDir, f"{date}-fps-{fps}-scene-{sceneId}-others.csv"))
tracksMeta = pd.read_csv(os.path.join(dataDir, f"{date}-fps-{fps}-scene-{sceneId}-meta.csv"))

In [6]:
visualizer = TrajectoryVisualizer()
colMapper = ColMapper(
        idCol='uniqueTrackId', 
        xCol='sceneX', 
        yCol='sceneY',
        xVelCol='sceneXVelocity', 
        yVelCol='sceneYVelocity', 
        speedCol='speed',
        xAccCol='sceneXAcceleration',
        yAccCol='sceneYAcceleration',
        accelerationCol='acceleration'
        
    
    )
transformer = TrajectoryTransformer(colMapper)
cleaner = TrajectoryCleaner(
    colMapper = colMapper,
    minSpeed = 0.0, # we can consider them as obstacles
    maxSpeed = 3.5,
    minAcceleration = -10.0,
    maxAcceleration = 10.0,
    minYDisplacement = minYDisplacement,
    maxXDisplacement = maxXDisplacement
)
transformer.deriveSpeed(pedDf)
transformer.deriveDisplacements(pedDf)
transformer.deriveAcceleration(pedDf)

transformer.deriveSpeed(pedDf2)
transformer.deriveDisplacements(pedDf2)
transformer.deriveAcceleration(pedDf2)

In [7]:
cleanPedDf = cleaner.cleanByYDisplacement(pedDf, byIQR=False)

# visualizer.show(pedDf, idCol = 'uniqueTrackId', xCol = 'sceneX', yCol = 'sceneY')
# visualizer.show(cleanPedDf, idCol = 'uniqueTrackId', xCol = 'sceneX', yCol = 'sceneY')
allPedIds = list(cleanPedDf["uniqueTrackId"].unique())
transformer.translateAllToLocalSource(cleanPedDf)

southIds = []
for pedId in allPedIds:
    trackDf = cleanPedDf[cleanPedDf["uniqueTrackId"] == pedId]
    trackMeta = tracksMeta[tracksMeta["uniqueTrackId"] == pedId].iloc[0]
    # print(trackMeta['verticalDirection'])
    if trackMeta['verticalDirection'] == "SOUTH":
        southIds.append(pedId)
        # print(trackMeta["uniqueTrackId"])
        X, Y = transformer.rotate(trackDf)
        cleanPedDf.loc[cleanPedDf["uniqueTrackId"] == pedId, 'localX'] = X
        cleanPedDf.loc[cleanPedDf["uniqueTrackId"] == pedId, 'localY'] = Y
len(cleanPedDf)


using min Y displacement (5)


7600

In [8]:
startingDf = cleanPedDf[cleanPedDf["displacementY"] < 1].copy()
midDf = cleanPedDf[(cleanPedDf["displacementY"] >= 1) & (cleanPedDf["displacementY"] < 4)].copy()
endDf = cleanPedDf[(cleanPedDf["displacementY"] >= 4)].copy()

list1 = [startingDf['speed'], midDf['speed'], endDf['speed']]

In [9]:
cleanPedDf.head()

,recordingId,frame,uniqueTrackId,sceneId,roadWidth,sceneX,sceneY,sceneXVelocity,sceneYVelocity,sceneXAcceleration,sceneYAcceleration,speed,displacementX,displacementY,acceleration,localX,localY
0,18,750,18023,210,5.0,0.629400,2.454854,-0.047612,0.481508,-0.044860,-0.028943,0.483857,0.000000,0.000000,0.053387,-0.000000,-0.000000
1,18,752,18023,210,5.0,0.637141,2.358691,-0.038708,0.480816,-0.044520,0.003463,0.482371,0.007742,0.096163,0.044655,-0.007742,0.096163
2,18,754,18023,210,5.0,0.644921,2.261374,-0.038896,0.486585,0.000940,-0.028846,0.488137,0.015521,0.193480,0.028862,-0.015521,0.193480
3,18,756,18023,210,5.0,0.653314,2.163065,-0.041969,0.491544,0.015368,-0.024797,0.493333,0.023915,0.291789,0.029173,-0.023915,0.291789
4,18,758,18023,210,5.0,0.661952,2.065082,-0.043190,0.489915,0.006105,0.008146,0.491815,0.032553,0.389772,0.010180,-0.032553,0.389772


In [11]:
uniqueTrackIds = cleanPedDf['uniqueTrackId'].unique()
len(uniqueTrackIds)

170

In [12]:
newData = { 'uniqueTrackId': uniqueTrackIds }
uniquePedDf = pd.DataFrame(newData)
uniquePedDf.head()

,uniqueTrackId
0,18023
1,18069
2,18090
3,18121
4,18134


In [16]:
uniquePedDf['startingTopSpeed'] = [0] * len(uniquePedDf)
uniquePedDf['midTopSpeed'] = [0] * len(uniquePedDf)
uniquePedDf['endTopSpeed'] = [0] * len(uniquePedDf)

uniquePedDf.head()

,uniqueTrackId,startingTopSpeed,midTopSpeed,endTopSpeed
0,18023,0,0,0
1,18069,0,0,0
2,18090,0,0,0
3,18121,0,0,0
4,18134,0,0,0


In [25]:
trackId = 18023
def putStartingTopSpeed(trackId):
    uniqueStartingDf = startingDf[startingDf['uniqueTrackId'] == trackId].copy() 
    return max(uniqueStartingDf['speed'])

def putMidTopSpeed(trackId):
    uniqueMidDf = midDf[midDf['uniqueTrackId'] == trackId].copy() 
    return max(uniqueMidDf['speed'])

def putEndTopSpeed(trackId):
    uniqueEndDf = endDf[endDf['uniqueTrackId'] == trackId].copy() 
    return max(uniqueEndDf['speed'])

print(putStartingTopSpeed(trackId), putMidTopSpeed(trackId), putEndTopSpeed(trackId))

0.4997371409051065 0.515984362650666 0.4783609332920143


In [ ]:
uniquePedDf['startingTopSpeed'] = 

18023 18069 18090 18121 18134 18180 18214 18225 18304 18311 18313 18382 19045 19055 19057 19070 19133 19199 19283 19295 19320 20043 20054 20083 20098 20121 20131 20219 20263 20279 20327 20362 20369 20376 20383 20475 21040 21063 21068 21115 21116 21125 21168 21169 21178 21185 21189 21190 21193 21223 21237 21285 21303 21404 21408 21413 21459 21472 21510 21518 22005 22133 22135 22151 22171 22184 22251 22297 22298 22451 22499 22516 23001 23053 23147 23156 23188 23235 23248 23249 23273 23274 23275 23391 23410 23431 23486 23514 24063 24064 24095 24122 24125 24132 24187 24291 24295 24297 24317 24420 25170 25205 25245 25348 25377 25407 25428 25468 25500 25501 25524 26070 26129 26138 26186 26192 26230 26249 26472 27047 27100 27119 27193 27288 27292 27304 27305 27343 27359 27361 27365 27384 27406 27425 27448 27488 27490 27492 27532 27536 27540 27567 28023 28055 28062 28090 28134 28211 28268 28320 28321 28351 28383 28422 28428 28452 28467 28474 29000 29211 29221 29235 29261 29266 29340 29353 2943